# 0. 코드 준비

In [13]:
import json
from pprint import pprint
import time
import datetime
import pandas as pd
import re

In [14]:
### DataBase ###
import pymysql
%reload_ext sql

# get key
with open('./key.json', 'r') as file:
    key = json.load(file)

# Connect to MySQL database
db = pymysql.connect(host = 'localhost', 
                     port = 3306, 
                     user='root', 
                     passwd = key['password'],
                     charset='utf8')

# Create a cursor object
cursor = db.cursor()
cursor.execute("USE lostark")

0

In [15]:
def strip_dict(d):
    dd = {}
    for k, v in d.items():
        if (type(k) == str) & (type(v) == str):
            dd[re.sub(k)] =  re.sub(v)
        elif type(v) == dict:
            v = strip_dict(v)
            dd[k] = v
        else:
            dd[k] = v
    return dd

In [24]:
# for notebook(for external work)
sql = "SELECT * FROM lostark.raw_character_data_table LIMIT 1;"
cursor.execute(sql)
columns = [col[0] for col in cursor.description]
columns

# for column in columns:
sql = f"SELECT characterCode, ArmoryEquipment FROM lostark.raw_character_data_table LIMIT 90000, 10000;"
cursor.execute(sql)
raw_data = pd.DataFrame(cursor.fetchall())
raw_data.to_csv(f'./ArmoryEquipment10.csv')

# 1. RAW_CHARACTER_DATA

>이름에 @가 들어가있으면 삭제? -> x : api호출은 정상, 전정실 검색은 있는 것과 없는 것이 있음
정지먹은 계정일 가능성



In [52]:
sql = "SELECT * FROM lostark.raw_character_data_table LIMIT 1;"
cursor.execute(sql)
columns = [col[0] for col in cursor.description]
column = columns[2]

sql = f"SELECT {column} FROM lostark.raw_character_data_table;"
cursor.execute(sql)
df = pd.DataFrame(cursor.fetchall(), columns=[column])

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570272 entries, 0 to 570271
Data columns (total 1 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   ArmoryProfile  570272 non-null  object
dtypes: object(1)
memory usage: 4.4+ MB


In [85]:
def get_all_keys(d):
    all_keys = set()
    if isinstance(d, dict):
        for k, v in d.items():
            all_keys.add(k)
            all_keys.update(get_all_keys(v))
    return tuple(all_keys)

all_keys = set(eval(eval(str(df.values[0]))[0]).keys())

for i in range(len(df)):
    d = eval(eval(str(df.values[i]))[0])
    all_keys.add(get_all_keys(d))
print(all_keys)

{'TownName', 'TotalSkillPoint', 'TownLevel', 'GuildMemberGrade', 'ItemMaxLevel', 'CharacterClassName', 'UsingSkillPoint', 'CharacterLevel', 'PvpGradeName', 'Title', 'ItemAvgLevel', 'GuildName', 'CharacterImage', ('TownName', 'CharacterLevel', 'TotalSkillPoint', 'ExpeditionLevel', 'CharacterImage', 'Title', 'TownLevel', 'GuildMemberGrade', 'ServerName', 'ItemMaxLevel', 'CharacterClassName', 'Tendencies', 'Stats', 'ItemAvgLevel', 'UsingSkillPoint', 'PvpGradeName', 'GuildName', 'CharacterName'), 'ServerName', 'ExpeditionLevel', 'Tendencies', 'Stats', 'CharacterName'}


In [27]:
# api key
lost_ark_api_key = key['api-key'] # apikey 마스킹
# header
headers = {'Authorization': 'Bearer {}'.format(lost_ark_api_key)}
timeout = 10
# api 분당 100회 제한
duration = 60 / 90

In [29]:
#! eval은 보안에 위험한 코드이나 현 단계에서는 넘어감(문자로 된 것을 코드화 하는 함수)
def eval_data(resp):
    edata = resp.text
    replace_text = ["null", "false", "true"]
    if any(rt in edata for rt in replace_text):
        edata = edata.replace("null", "None")
        edata = edata.replace("false", "False")
        edata = edata.replace("true", "True")
    edata = eval(edata)
    return edata

# response preprocessing
def text_cleaner(t):
    br_cleaner = re.compile("<br>", re.I)
    html_cleaner = re.compile("<.*?>[ ]?")
    escape_cleaner = re.compile("\\\[rn]|\s\s")
    cleantext = re.sub(br_cleaner, " ", t)
    cleantext = re.sub(html_cleaner, "", cleantext)
    cleantext = re.sub(escape_cleaner, "", cleantext)
    cleantext = cleantext.strip()
    return cleantext

def get_total_profile_responses(characterName):
    url = 'https://developer-lostark.game.onstove.com/armories/characters/{}'.format(characterName) 
    resp = requests.get(url=url, headers=headers, timeout=timeout, verify = False)
    resp = eval_data(resp)
    resp = text_cleaner(str(resp))
    profiles_responses = ast.literal_eval(resp)
    time.sleep(duration)
    return profiles_responses

In [33]:
cdf = df[df[column].str.contains('@')]
cdf

,characterName
1018,11531513@78901137
1120,12121255515@150081990
1235,123123ASD123@69663327
1269,12312666@150082007
1272,12312easdasd@98629403
...,...
562624,후돈@4968166
566406,흠슴흠@71332096
568066,히쿨@150916039
568583,힐딱이따위안해@136113351


In [51]:
for i in cdf.values:
    print(i)

['11531513@78901137']
['12121255515@150081990']
['123123ASD123@69663327']
['12312666@150082007']
['12312easdasd@98629403']
['1232131457@90354026']
['123asdf3432@98122116']
['12412414126@150004655']
['12AS3@111554637']
['12ase12as@117334619']
['12ES@111555520']
['13fdg4@72259220']
['147@57796405']
['15312@150035595']
['15348@150035444']
['15351@150035811']
['1ngsu@149971099']
['222xxx@136427398']
['23423432432@150046504']
['2342343245@150046563']
['234234543@150046396']
['234342432@150046830']
['24k뿅망치@62853733']
['25235@122475227']
['32twssd@150040665']
['3435DGFYR6@69665797']
['345GFGF@69666981']
['346343435@122474873']
['35t23ytgg@150040571']
['45345456@150036215']
['456781234@150036412']
['4786415@150036012']
['4DIOS@138795360']
['4번타자김효섭@149660741']
['51515999@150082028']
['5지고@85154160']
['676543243@136887761']
['6칸다채웟다@123354448']
['6칸채우기@123343171']
['6칸채우기2@123348386']
['7436274@150004695']
['82tk@70999949']
['963@57810671']
['96년생김혀웃@144299112']
['AdiosFRIEND@127379925']
['agd

In [47]:
get_total_profile_responses(cdf[column].values[0])

{'ArmoryProfile': {'CharacterImage': None,
  'ExpeditionLevel': 0,
  'PvpGradeName': None,
  'TownLevel': None,
  'TownName': '컨텐츠 개방 필요',
  'Title': None,
  'GuildMemberGrade': None,
  'GuildName': None,
  'UsingSkillPoint': 0,
  'TotalSkillPoint': 0,
  'Stats': None,
  'Tendencies': [{'Type': '지성', 'Point': 0, 'MaxPoint': 1000},
   {'Type': '담력', 'Point': 0, 'MaxPoint': 1000},
   {'Type': '매력', 'Point': 0, 'MaxPoint': 1000},
   {'Type': '친절', 'Point': 0, 'MaxPoint': 1000}],
  'ServerName': '',
  'CharacterName': '11531513@78901137',
  'CharacterLevel': 1,
  'CharacterClassName': '전사(남)',
  'ItemAvgLevel': '0.00',
  'ItemMaxLevel': '0.00'},
 'ArmoryEquipment': None,
 'ArmoryAvatars': None,
 'ArmorySkills': None,
 'ArmoryEngraving': None,
 'ArmoryCard': None,
 'ArmoryGem': None,
 'ColosseumInfo': {'Rank': 0,
  'PreRank': 0,
  'Exp': 0,
  'Colosseums': [{'SeasonName': '프리 시즌',
    'Competitive': None,
    'TeamDeathmatch': None,
    'Deathmatch': None,
    'TeamElimination': None,
    '